# The Agentic Loop

In this section, we'll explore what an agentic loop is and why it's essential for building reliable, production-ready AI agents.

You'll learn:
- What makes an AI system "agentic"
- How the event loop pattern enables autonomous decision-making
- Why Temporal is the ideal platform for building durable AI agents
- How to observe an agent making real-time decisions

This is a **conceptual exploration** with a working demonstration—the focus is on understanding the patterns and principles, not on writing code.

## Setup Your Notebook

Run the following code blocks to install various packages and tools necessary to run this notebook.

In [1]:
# Mermaid renderer, run at the beginning to setup rendering of diagrams
import base64
from IPython.display import Image, display

def render_mermaid(graph_definition):
    """
    Renders a Mermaid diagram in Google Colab using mermaid.ink.

    Args:
        graph_definition (str): The Mermaid diagram code (e.g., "graph LR; A-->B;").
    """
    graph_bytes = graph_definition.encode("ascii")
    base64_bytes = base64.b64encode(graph_bytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

## So far in this workshop...

In the previous sections of this workshop, we've built a research application that:

1. **Accepts user input** - Takes a research topic from the user
2. **Calls the LLM for research** - Uses an LLM to generate research content
3. **Generates PDF** - Creates a formatted output document
4. **Returns success/failure** - Provides execution results
5. **Accepts Signals and Queries** - Enables external interaction with running Workflows

In [2]:
diagram = """
graph LR
    A[User calls LLM]
    signals[signals] --> process[ ]
    A --> process
    process --> B[Generating a PDF from LLM response]
    queries[queries] --> process
"""
render_mermaid(diagram)

## From Simple Chain to Agentic Loop

But what if we wanted to make it more complex? What if we wanted our research application to:

- **Decide if it needs more information** after an initial web search
- **Execute multiple searches** based on what it learns
- **Analyze results** and determine if they're sufficient

This is where the **agentic loop** becomes essential.

## The Key Difference

### What We Built Before: Chain Workflow

**User Input → LLM Call → Generate PDF → Done**

This is a **linear process**. The **user** determines when we're done.

In [3]:
diagram = """
graph LR
    A[User calls LLM] --> B[Generating a PDF from LLM response]
"""
render_mermaid(diagram)

### With an Agentic Loop: The Agent Decides

At a high level, AI agents have an **event loop**, and in that event loop, we make calls out to the LLM to ask it for directions. **The LLM is what drives the flow** of the application.

We might invoke some downstream actions (e.g., make microservice requests, query databases, call external APIs). Then we may consult with the user, and then go through the loop again.

In the loop, it:
- **Plans**: The LLM analyzes the current state and decides what to do next
- **Executes**: The agent calls relevant tools or APIs
- **Observes**: Reviews results and updates context
- **Repeats** until the LLM hits its goal or the user stops it

<img src="https://i.postimg.cc/1zhMS6YL/Loop.png" width="300"/>

## Comparison: Chain vs Agentic Loop

| Aspect | Chain Workflow | Agentic Loop |
|--------|---------------|---------------|
| **Decision Maker** | Developer (hardcoded) | LLM (runtime decisions) |
| **Execution Path** | Fixed, predetermined | Dynamic, adapts to context |
| **Iterations** | Single pass through | Multiple iterations possible |
| **When It Stops** | After last step completes | When goal achieved or max iterations hit |

## What is an AI Agent?

An **AI agent** is an autonomous system that pursues goals through **continuous decision-making and action**.

Think of an AI agent as an autonomous system that doesn't just respond once, but **continuously** works toward achieving a specific goal. Unlike traditional software that follows predetermined steps, an AI agent makes **decisions dynamically** based on the current situation and available information.

## Real-World Example: Customer Support Agent

Imagine a customer contacts support saying: *"My order hasn't arrived, I need a refund immediately!"*

A traditional application would follow a rigid flowchart. But an AI agent operates differently:

**Goal**: Resolve the customer's issue and retain their business

**The agent's loop in action**:

| Step | Phase | What Happens |
|------|-------|-------------|
| 1 | **Plan** | "I should first check the order status" |
| 2 | **Execute** | Queries order database |
| 3 | **Observe** | Package is in transit, arriving tomorrow |
| 4 | **Plan** | "Customer needs it urgently, but it's arriving soon—offer expedited upgrade" |
| 5 | **Execute** | Checks if expedited shipping is available |
| 6 | **Observe** | Next-day air available for $15 |
| 7 | **Plan** | "Customer lifetime value is $500—offer free upgrade" |
| 8 | **Execute** | Sends free expedited shipping offer |
| 9 | **Observe** | Customer accepts offer |
| 10 | **Goal Achieved** | Issue resolved, customer retained ✓ |

Notice how the agent didn't follow a pre-programmed "if customer says refund, then refund" rule. It **autonomously decided** the best path based on the specific situation.

## The Agent's Limitation

Our customer support agent has a goal: "resolve dispute and retain customer." But here's the problem—**an LLM by itself can only generate text**.

It can think through the situation and decide what *should* happen next:
- "I should check the order status"
- "I need to see if the customer has any previous complaints"
- "I should offer a refund"

But it can't actually *do* any of those things.

**This is where tools come in.**

## What is a Tool?

A **tool** is an external function that AI systems can call to perform specific tasks beyond just generating text. Tools take real actions in the world—they might:
- Query databases
- Call external APIs
- Search for information
- Perform calculations
- Book reservations
- Send emails
- Generate files

### How Agents Use Tools

Agents are **aware of the tools** they have available while attempting to achieve their goal. The agent:
1. Evaluates which tools are available
2. Decides which tool will help progress toward the goal
3. Executes the tool
4. Observes the result
5. Decides the next action based on what it learned

## How Agents Execute Tools: The Loop in Action

Each iteration of the agentic loop follows this pattern:

1. **LLM decides what to do** → "I need to check the order status"
2. **Agent prepares the tool call** → Formats the request: `check_order(order_id="12345")`
3. **Tool executes** → Queries the database and returns: `{status: "shipped", tracking: "1Z999..."}`
4. **Agent updates context** → Adds the result to the conversation history
5. **Loop continues** → LLM sees the new information and decides the next action

This continues until the LLM determines the goal is achieved.

In [4]:
diagram = """
graph LR
    Goal[the goal] --> LLM[LLM<br/>makes decisions on<br/>what to do next]
    LLM --> Tool[Tool<br/>does what the LLM<br/>decided]
    Tool --> LLM
"""
render_mermaid(diagram)

## Core Characteristics of AI Agents

What makes an application agentic?

### 1. Goal-Oriented
It has a clear objective (e.g., generate a research report, book a flight, resolve a customer issue)

### 2. Tool Usage
It combines multiple capabilities (LLM reasoning + external actions like API calls, database queries, or PDF generation)

### 3. Autonomous Decision Making
The LLM decides how to approach the problem and which tools to use

**Important**: Not all agents require loops! Simple agents can be goal-oriented with tool usage and autonomous decision-making without requiring iterative loops. However, **agentic loops** enable more sophisticated behaviors where the agent can reassess and adapt.

## Why Agentic Loops Are Harder Than Chains

### 2. Complex State Management
- **Chain**: Each step only needs output from previous step
- **Loop**: Must track entire conversation history, tool results, intermediate decisions

### 2. Expensive Retries
- **Chain**: If step 3 fails, retry step 3
- **Loop**: If iteration 12 fails, it's pricy to redo iterations 1-11 (each might have called expensive APIs or LLMs)

### 3. Non-Linear Execution
- **Chain**: Always the same sequence
- **Loop**: Different execution paths based on what the agent discovers

### 4. Long-Running Processes
- **Chain**: Might take seconds or minutes
- **Loop**: Could run for hours or days (e.g., waiting for user approval between iterations)

This is where **Temporal Workflows** come in.

## Demo (Expand for instructor notes or to run on your own)
<!--
1. Clone this repository: `https://github.com/temporalio/edu-ai-workshop-agentic-loop`. The instructions will also be in the README.
2. We will now showcase how an AI agent can dynamically choose its own tools with Temporal. Route to the `module_one_04_agentic_loop` directory with `cd module_one_04_agentic_loop`.
3. Open three terminal windows.
4. In one terminal window, start the Temporal server with `temporal server start-dev --ui-port 8080 --db-filename clusterdata.db` (if not already running from previous demos).
5. In another terminal window, navigate to the `module_one_04_agentic_loop` directory and run the worker with `python worker.py`. You'll see output indicating the Worker is listening on the task queue.
6. In the third terminal window, execute your Workflow with `python starter.py`.
7. You'll be prompted to enter a goal for the AI agent (e.g., "Book a flight from RDU to London on November 18").
8. Once the Workflow Execution completes, point out on the Web UIL
    - Input, output
    - The execution of each tool (search_flights, check_availability, book_flight)
    - The difference in the tools selected if you do something like "Book a flight from RDU to NYC on Oct 18, check seat availability first and show me the total cost" vs. "Book a flight from RDU to NYC on Oct 18".
    
-->

## Real-World Agent Architecture: Agents Calling Agents

In production systems, complexity multiplies when agents coordinate with other agents.

### Example: Research Report Generation System

Your "Research Agent" might:
1. Call a **Web Scraper Agent** to gather sources (scrapes 5 research websites)
2. Call a **Fact Checker Agent** to verify claims (validates statistics against government data)
3. Call a **Citation Generator Agent** to format references
4. Call a **PDF Generator Agent** (what you built!)

Each of these agents has its own loop:
- **Web Scraper Agent**: Plan where to search → Execute scrape → Observe quality → Decide if more sources needed
- **Fact Checker Agent**: Plan which claims to verify → Execute verification → Observe results → Decide confidence level

<img src="https://i.postimg.cc/KzH2vx3y/agents-calling-agents.png" width="500"/>

### Failure Points Multiply

Now consider failure scenarios:
- Network partition between Research Agent and Web Scraper Agent
- Web Scraper Agent's 3rd iteration times out
- Fact Checker Agent finds contradictory data and needs human review
- PDF Generator runs out of memory

**Without durability**: Any failure at any level cascades, and you lose all work across all agents.

**With Temporal**: Each agent's loop is independently durable. Failures are isolated and recoverable.

## The Agentic Loop in Temporal

In Temporal, an agentic loop is represented as a **Workflow**. 

### How Temporal Enables Durable Agents

When you build an agentic loop with Temporal:

#### 1. Every step is recorded
If you trigger an LLM invocation, then execute a tool, and finally hand off control to another agent, Temporal records every step of that process in its event history.

#### 2. Automatic recovery
If something goes wrong—a crash, network glitch, or service interruption—Temporal automatically replays the Workflow from the beginning, using the recorded event history to skip already-completed steps.

#### 3. No data loss
Temporal persistently records application progress and stores all Activity results in its durable event log.

#### 4. No duplicate work
Once an Activity completes successfully, its result is recorded. On replay, Temporal returns the stored result instead of re-executing the Activity.

## Key Takeaways

### What You Learned Today

1. **Agentic loops** are fundamentally different from chain workflows:
   - Dynamic execution paths
   - Unknown number of iterations
   - LLM-driven decision making

2. **Agents require three core capabilities**:
   - Goal-oriented behavior
   - Tool usage
   - Autonomous decision making

3. **The agentic loop pattern**: Plan → Execute → Observe → Repeat

4. **Durability is critical** for agentic loops:
   - Prevents expensive re-work
   - Preserves state across long waits
   - Enables reliable multi-agent coordination
   - Saves significant costs at scale

5. **Temporal makes agentic loops production-ready**:
   - Every decision is recorded
   - Automatic recovery from failures
   - No duplicate work
   - Simple code, complex reliability

## What's Next?

This workshop introduced you to the **concept** of the importance of foundations of AI, human in the loop, agentic loops and why durability matters.

### Resources

- [Temporal Docs](https://docs.temporal.io/)
- [Code Repository](https://github.com/temporalio/edu-ai-workshop-agentic-loop)
- [Free Temporal 101 Course](https://learn.temporal.io/courses/temporal_101/)

### Thank You!

Please leave your feedback [here](https://docs.google.com/forms/d/e/1FAIpQLSfkHMev6KCNGFHpVNydyjgAh2ALeHNVYv9TaSrAoBsT0KmNHQ/viewform?usp=header)!